In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
import json

In [ ]:
df_bohdan = pd.read_csv('final_dataset.csv')
df_sofiia = pd.read_csv('rdi_lonlat.csv')
df_names = pd.read_csv('archive-diploma-kaggle/all_dist_villages.csv')

In [ ]:
df_names = df_names[["admin4Name", "admin4Na_1", "admin4Pcod"]]

In [ ]:
df_orig = df_bohdan.merge(df_sofiia, on='admin4Pcod', how='inner', suffixes=('_1', ''))
df_orig = df_orig.merge(df_names, on='admin4Pcod', how='inner', suffixes=('_1', ''))

In [ ]:
df = df_orig[['lon', 'lat', 'RDI', 'admin4Name', "admin4Pcod"]]
df = df.rename(columns={"admin4Pcod": "Postal code", "admin4Name": "Locality"})

df['RDI'] = np.round(df['RDI'], 3)

bins = [0, 0.65, 0.75, 0.85, 0.90, 1.00]
labels = ['Very Low', 'Low', 'Moderate', 'High', 'Very High']
labels_legend = ['0.00-0.65 (Very Low)', '0.65-0.75 (Low)', '0.75-0.85 (Moderate)', '0.85-0.90 (High)', '0.90-1.00 (Very High)']
df['RDI category'] = pd.cut(df['RDI'], bins=bins, labels=labels, include_lowest=True)
df['RDI legend'] = pd.cut(df['RDI'], bins=bins, labels=labels_legend, include_lowest=True)
df.sort_values('RDI', ascending=False, inplace=True)

custom_data = df[['Locality', "Postal code", 'RDI']]

In [ ]:
center = (0.5 * (df['lon'].max() + df['lon'].min()), 0.5 * (df['lat'].max() + df['lat'].min()))
print(center)

In [ ]:
def plot_borders(dst_fig):
    fig = go.Figure()

    # Load GeoJSON geometry data in X, Y format
    with open('occupation.geojson') as f:
        geojson = json.load(f)

    counter = 0
    # Loop through each feature in the GeoJSON
    for feature in geojson['features']:
        # Extract the coordinates for each polygon
        coordinates = feature['geometry']['coordinates']
        
        # If the geometry is a polygon
        if feature['geometry']['type'] == 'Polygon':
            # Extract the outer ring of the polygon
            x_coords, y_coords = zip(*coordinates[0])  # Only take the first ring
            
            # Add the polygon's boundary as lines to the figure
            fig.add_trace(go.Scattermapbox(
                fill="toself",
                lon=x_coords,
                lat=y_coords,
                mode='lines',  # Lines only
                line=dict(color='black'),  # Color of the lines
                hoverinfo='none',
                fillcolor="rgba(255, 0, 0, 0.5)",
                name = "Occupation zone",
                showlegend= True if counter == 0 else False,
                legendgroup="1"
            ))

            counter += 1

        # If the geometry is a MultiPolygon
        elif feature['geometry']['type'] == 'MultiPolygon':
            for polygon in coordinates:
                x_coords, y_coords = zip(*polygon[0])  # Take the first ring of each polygon
                
                # Add the polygon's boundary as lines to the figure
                fig.add_trace(go.Scattermapbox(
                    fill="toself",
                    lon=x_coords,
                    lat=y_coords,
                    mode='lines',  # Lines only
                    line=dict(color='black'),  # Color of the lines
                    hoverinfo='none',
                    fillcolor="rgba(255, 0, 0, 0.25)",
                    name = "Occupation zone",
                    showlegend= True if counter == 0 else False,
                    legendgroup="1"
                ))

                counter += 1

    # Remove interactive elements
    fig.update_traces(hoverinfo='none', selector=dict(type='scatter'))  # Disable hover info

    for trace in fig.data:
        dst_fig.add_trace(trace)

In [ ]:
colors = { '0.00-0.65 (Very Low)': '#d7191c', '0.65-0.75 (Low)': '#f5a153', '0.75-0.85 (Moderate)': '#f7ff0a', '0.85-0.90 (High)': '#8acc62', '0.90-1.00 (Very High)': '#1a9641' }

fig = px.scatter_mapbox(df, lon='lon', lat='lat', color='RDI legend', color_discrete_map=colors, hover_data={
        # 'lon': False,  # Hide X
        # 'lat': False,  # Hide Y
        'RDI category': True,  # Hide RDI category
        'Locality': True,  # Show RDI (continuous data)
        'Postal code': True,  # Show Category
        'RDI': True  # Show Description
    },
    mapbox_style='open-street-map',
    zoom=5,
    center={
        'lon': center[0],
        'lat': center[1]
    },)
    # width=976,
    # height=686)

hovertemplate = "<br>".join([
    "<b>%{customdata[1]}</b>",
    "%{customdata[2]}",
    "RDI: %{customdata[3]} (%{customdata[0]})<extra></extra>",
])

fig.update_traces(
    marker=dict(size=6),
    hovertemplate=hovertemplate,
    # showlegend=False,
)

fig.update_layout(
    # title='Sample scatter plot',
    plot_bgcolor="white",
    xaxis_title=None,
    xaxis_showticklabels=False,
    yaxis_title=None,
    yaxis_showticklabels=False,
    legend=dict(
        title_text='Rural development index',
        title_font=dict(size=16, color='black'),
        x=0.1,  # Position x at the left
        y=0.1,  # Position y at the bottom
        xanchor='left',  # Anchor the x position to the left
        yanchor='bottom',  # Anchor the y position to the bottom
        orientation='v',
        bgcolor='rgba(255, 255, 255, 0.75)',  # Transparent background
        bordercolor='rgba(255, 255, 255, 0)',  # Optional: remove border
        itemsizing='constant'
    ),
)

fig.update_xaxes(showgrid=False, zeroline=False)
fig.update_yaxes(showgrid=False, zeroline=False)
fig['layout'].update(margin=dict(l=0,r=0,b=0,t=24))

plot_borders(fig)

config = {'scrollZoom': True}

fig.show(config=config)

fig.write_html('rdi_mcdm_interactive_occupated.html', config=config)